In [1]:
## Post-Proc - Reject Option Classifier
%matplotlib inline
import sys
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
# Import IBM's AI Fairness tooolbox
from aif360.datasets      import BinaryLabelDataset
from aif360.datasets      import AdultDataset, GermanDataset, CompasDataset
from aif360.metrics       import ClassificationMetric, BinaryLabelDatasetMetric
from aif360.metrics.utils import compute_boolean_conditioning_vector
from aif360.algorithms.postprocessing.reject_option_classification import RejectOptionClassification
from common_utils         import compute_metrics
# Import scikit-learn core slibraries
from sklearn.model_selection import train_test_split
from sklearn.ensemble        import RandomForestClassifier
from sklearn.linear_model    import LogisticRegression
from sklearn.preprocessing   import StandardScaler,MinMaxScaler
from sklearn.metrics         import accuracy_score
from sklearn.metrics         import balanced_accuracy_score
from tqdm import tqdm
from IPython.display   import Markdown, display
from ipywidgets import interactive, FloatSlider
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read the cleaned Taiwan-Credit-Card-Cleaned-Marriage dataset
Taiwan_df = pd.read_csv('./input/Taiwan-Credit-Card-Cleaned-GENDER.csv')

In [3]:
# Set privileged (1)/ unprivileged (0)/ favourable (1) / unfavourable values (0)
protected_attr      = 'GENDER'
priv_grp            = 1  # Males 
unpriv_grp          = 0  # Females  
lab                 = 'DEFAULT'
fav_label           = 1 # Will not default next month
unfav_label         = 0 # Will default next month
privileged_groups   = [{protected_attr: priv_grp}]   # Males
unprivileged_groups = [{protected_attr: unpriv_grp}] # Females
# Upper and lower bound on the fairness metric used
metric_ub = 0.05
metric_lb = -0.05
allowed_metrics = ["Statistical parity difference",
                   "Average odds difference",
                   "Equal opportunity difference"]
metric_name = "Statistical parity difference"

In [4]:
# Create the features and label splits
X = Taiwan_df.drop(lab,axis=1)
y = Taiwan_df[lab]

In [5]:
# Create a Binary Label Dataset to use with AIF360 APIs
Taiwan_bld = BinaryLabelDataset(df=pd.concat((X, y), axis=1),
                                label_names=[lab], protected_attribute_names=[protected_attr],
                                favorable_label=fav_label, unfavorable_label=unfav_label)

In [6]:
# Get the dataset and split into train and test
Taiwan_orig_train, Taiwan_orig_vt   = Taiwan_bld.split([0.8])    # , shuffle=True)
Taiwan_orig_valid, Taiwan_orig_test = Taiwan_orig_vt.split([0.5])# , shuffle=True)

In [7]:
# First test whether it is possible to predict the Sensitive Variable from the training dataset
Taiwan_train_df, d = Taiwan_orig_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Taiwan_test_df, d  = Taiwan_orig_vt.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
X_se_train = Taiwan_train_df.drop(protected_attr,axis=1)
y_se_train = Taiwan_train_df[protected_attr]
X_se_test  = Taiwan_test_df.drop(protected_attr,axis=1)
y_se_test  = Taiwan_test_df[protected_attr]

Se_BiasedLogModel = LogisticRegression(random_state=101)
Se_BiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
Se_BiasedLogModel.fit(X_se_train, y_se_train) 
Se_BiasedRfcModel.fit(X_se_train, y_se_train) 

yseLog_pred =  Se_BiasedLogModel.predict(X_se_test)
yseRfc_pred =  Se_BiasedRfcModel.predict(X_se_test)
# Now test whether we can predict Gender from the test dataset
print('Model Accuracy for predicting the Sensitive Variable before bias transformation:')
print(f"Biased Logistic regression validation accuracy: {Se_BiasedLogModel.score(X_se_test, y_se_test)}")
print(f"Biased Random Forest       validation accuracy: {Se_BiasedRfcModel.score(X_se_test, y_se_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_se_test, yseLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_se_test, yseRfc_pred)}")

Model Accuracy for predicting the Sensitive Variable before bias transformation:
Biased Logistic regression validation accuracy: 0.5065
Biased Random Forest       validation accuracy: 0.5156666666666667

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.5008655556202553
Biased Random Forest       balanced accuracy  : 0.5102468717092996


In [8]:
metric_test_bld = BinaryLabelDatasetMetric(Taiwan_orig_test,
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)
print('Number of instances           :', metric_test_bld.num_instances())
print('Base Rate                     :', metric_test_bld.base_rate())
print('Consistency                   :', metric_test_bld.consistency())
print('Disparate Impact              :', metric_test_bld.disparate_impact())
print('Mean Difference               :', metric_test_bld.mean_difference())
print('Statistical Parity Difference :', metric_test_bld.statistical_parity_difference()) 

Number of instances           : 3000.0
Base Rate                     : 0.779
Consistency                   : [0.73886667]
Disparate Impact              : 1.0691643472561294
Mean Difference               : 0.05197150525354044
Statistical Parity Difference : 0.05197150525354044


In [9]:
### Train logistic regression and random forest classifier on the training data

In [10]:
#Train Logistic Regression and Random Forest Classifiers on original training data
scale_orig          = MinMaxScaler()
X_train             = scale_orig.fit_transform(Taiwan_orig_train.features)
y_train             = Taiwan_orig_train.labels.ravel()

BiasedLogModel      = LogisticRegression(random_state=101)
BiasedRfcModel      = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)

BiasedLogModel.fit(X_train, y_train)
BiasedRfcModel.fit(X_train, y_train)

BiasedLogPrediction = BiasedLogModel.predict(X_train)
BiasedRfcPrediction = BiasedRfcModel.predict(X_train)

# Determine the baseline model accuracy for Logistic Regression and Random Forest Classifiers
display(Markdown("#### Biased training dataset - model scores"))
print('Biased Logistic regression validation accuracy:', BiasedLogModel.score(X_train, y_train))
print('Biased Random Forest       validation accuracy:', BiasedRfcModel.score(X_train, y_train))
print('')
print('Biased Logistic regression balanced accuracy  :', balanced_accuracy_score(y_train, BiasedLogPrediction))
print('Biased Random forest balanced accuracy        :', balanced_accuracy_score(y_train, BiasedRfcPrediction))

#### Biased training dataset - model scores

Biased Logistic regression validation accuracy: 0.807
Biased Random Forest       validation accuracy: 0.8096666666666666

Biased Logistic regression balanced accuracy  : 0.6007241878750259
Biased Random forest balanced accuracy        : 0.6180156352612024


In [11]:
# Create a numpy array pos_ind of all the '1s' (positive class index) to use for the test data.   
# Update a COPY of the training dataset labels with the predicted labels. 
LogTrain_pred        = Taiwan_orig_train.copy(deepcopy=True)
RfcTrain_pred        = Taiwan_orig_train.copy(deepcopy=True)
LogTrain_pred.labels = BiasedLogPrediction
RfcTrain_pred.labels = BiasedRfcPrediction
# At this point we have copies of the Training dataset, whose labels are modified by 
# the Logistic Regression and Random Forest classifier predictions respectively.

In [12]:
### Use BiasedLogModel and BiasedRfcModel on the test and validation data and get scores

In [13]:
# Create a numpy array pos_ind of all the '1s' (positive class index) to use for the test data.   
pos_log_ind = np.where(BiasedLogModel.classes_ == Taiwan_orig_valid.favorable_label)[0][0]
pos_rfc_ind = np.where(BiasedRfcModel.classes_ == Taiwan_orig_valid.favorable_label)[0][0]

# Obtain scores for Validation sets using the BiasedLogModel
LogValid_pred = Taiwan_orig_valid.copy(deepcopy=True) # Create a copy of the original Validation dataset
RfcValid_pred = Taiwan_orig_valid.copy(deepcopy=True) # Create a copy of the original Validation dataset

X_log_valid = scale_orig.transform(LogValid_pred.features)
y_log_valid = LogValid_pred.labels

X_rfc_valid = scale_orig.transform(RfcValid_pred.features)
y_rfc_valid = RfcValid_pred.labels

# Update a COPY of the validation dataset labels with the predicted labels.
LogValid_pred.scores = BiasedLogModel.predict_proba(X_log_valid)[:,pos_log_ind].reshape(-1,1)
RfcValid_pred.scores = BiasedRfcModel.predict_proba(X_rfc_valid)[:,pos_rfc_ind].reshape(-1,1)
# At this point we have copies of the Validation dataset, whose labels are modified by 
# the Logistic Regression and Random Forest classifier predictions respectively.

In [14]:
# Create a numpy array pos_ind of all the '1s' (positive class index) to use for the test data.   
pos_log_indx = np.where(BiasedLogModel.classes_ == Taiwan_orig_test.favorable_label)[0][0]
pos_rfc_indx = np.where(BiasedRfcModel.classes_ == Taiwan_orig_test.favorable_label)[0][0]

# Obtain scores for the Test dataset using the BiasedLogModel
LogTest_pred = Taiwan_orig_test.copy(deepcopy=True) # Create a copy of the original Test dataset
RfcTest_pred = Taiwan_orig_test.copy(deepcopy=True) # Create a copy of the original Test dataset

X_log_test   = scale_orig.transform(LogTest_pred.features)
y_log_test   = LogTest_pred.labels

X_rfc_test   = scale_orig.transform(RfcTest_pred.features)
y_rfc_test   = RfcTest_pred.labels

# Update a COPY of the Test dataset labels with the predicted labels.
LogTest_pred.scores = BiasedLogModel.predict_proba(X_log_test)[:,pos_log_indx].reshape(-1,1)
RfcTest_pred.scores = BiasedRfcModel.predict_proba(X_rfc_test)[:,pos_rfc_indx].reshape(-1,1)
# At this point we have copies of the Test dataset, whose labels are modified by 
# the Logistic Regression and Random Forest classifier predictions respectively.

In [15]:
# At this point we have run the Test dataset through both classifiers.
# These are the counts of the labels that have been predicted..
LogTest_df,d = LogTest_pred.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
RfcTest_df,d = RfcTest_pred.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
print('Test Before - Logistic Regression - DEFAULT value counts:')
print(LogTest_df.DEFAULT.value_counts())
print('Test Before - Random Forest       - DEFAULT value counts:')
print(RfcTest_df.DEFAULT.value_counts())

Test Before - Logistic Regression - DEFAULT value counts:
1.0    2337
0.0     663
Name: DEFAULT, dtype: int64
Test Before - Random Forest       - DEFAULT value counts:
1.0    2337
0.0     663
Name: DEFAULT, dtype: int64


In [16]:
# Find the optimal parameters from the Validation set using Logistic Regression trained datasets
# Best threshold for classification only (no fairness)
num_thresh       = 100
ba_log_arr       = np.zeros(num_thresh) # Set up a Balanced Accuraccy array for 100 entries
ba_rfc_arr       = np.zeros(num_thresh) # Set up a Balanced Accuraccy array for 100 entries
class_log_thresh_arr = np.linspace(0.01, 0.99, num_thresh)
class_rfc_thresh_arr = np.linspace(0.01, 0.99, num_thresh)

for idx_log, class_log_thresh in enumerate(class_log_thresh_arr):
    # For the Logistic Regression Classifier trained model..
    fav_log_inds = LogValid_pred.scores > class_log_thresh # If the predicted score > threshold, store in fav_ind 
    LogValid_pred.labels[fav_log_inds]  = LogValid_pred.favorable_label
    LogValid_pred.labels[~fav_log_inds] = LogValid_pred.unfavorable_label
    classified_metric_log_valid = ClassificationMetric(Taiwan_orig_valid, LogValid_pred, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups  =privileged_groups)
    ba_log_arr[idx_log] = 0.5*(classified_metric_log_valid.true_positive_rate()\
                              +classified_metric_log_valid.true_negative_rate())

for idx_rfc, class_rfc_thresh in enumerate(class_rfc_thresh_arr):
    # Do the same for the Random Forest Classifier trained model..
    fav_rfc_inds = RfcValid_pred.scores > class_rfc_thresh # If the predicted score > threshold, store in fav_ind 
    RfcValid_pred.labels[fav_rfc_inds]  = RfcValid_pred.favorable_label
    RfcValid_pred.labels[~fav_rfc_inds] = RfcValid_pred.unfavorable_label
    classified_metric_rfc_valid = ClassificationMetric(Taiwan_orig_valid, RfcValid_pred, 
                                             unprivileged_groups=unprivileged_groups,
                                             privileged_groups  =privileged_groups)
    ba_rfc_arr[idx_rfc] = 0.5*(classified_metric_rfc_valid.true_positive_rate()\
                              +classified_metric_rfc_valid.true_negative_rate())
    
# Outside the for loop..
best_log_ind          = np.where(ba_log_arr == np.max(ba_log_arr))[0][0]
best_log_class_thresh = class_log_thresh_arr[best_log_ind]

best_rfc_ind          = np.where(ba_rfc_arr == np.max(ba_rfc_arr))[0][0]
best_rfc_class_thresh = class_rfc_thresh_arr[best_rfc_ind]

display(Markdown("#### for the Validation set"))
print("Best Logistic Regression balanced accuracy (no fairness constraints)           = %.4f" % np.max(ba_log_arr))
print("Optimal logistic regression classification threshold (no fairness constraints) = %.4f" % best_log_class_thresh)
print('')
print("Best Random Forest balanced accuracy (no fairness constraints)                 = %.4f" % np.max(ba_rfc_arr))
print("Optimal Random Forest classification threshold (no fairness constraints)       = %.4f" % best_rfc_class_thresh)

#### for the Validation set

Best Logistic Regression balanced accuracy (no fairness constraints)           = 0.6990
Optimal logistic regression classification threshold (no fairness constraints) = 0.7425

Best Random Forest balanced accuracy (no fairness constraints)                 = 0.7224
Optimal Random Forest classification threshold (no fairness constraints)       = 0.7722


In [17]:
# Estimate optimal parameters for the ROC method
# Create an instance of the ROC classification model
ROC_log = RejectOptionClassification(unprivileged_groups= unprivileged_groups, 
                                 privileged_groups  = privileged_groups, 
                                 low_class_thresh   = 0.01, 
                                 high_class_thresh  = 0.99,
                                 num_class_thresh   = 100, 
                                 num_ROC_margin     = 50,
                                 metric_name        = metric_name,
                                 metric_ub          = metric_ub, 
                                 metric_lb          = metric_lb)
ROC_rfc = RejectOptionClassification(unprivileged_groups= unprivileged_groups, 
                                 privileged_groups  = privileged_groups, 
                                 low_class_thresh   = 0.01, 
                                 high_class_thresh  = 0.99,
                                 num_class_thresh   = 100, 
                                 num_ROC_margin     = 50,
                                 metric_name        = metric_name,
                                 metric_ub          = metric_ub, 
                                 metric_lb          = metric_lb)
# Train the ROC model on the validation dataset 
ROC_log = ROC_log.fit(Taiwan_orig_valid, LogValid_pred)
ROC_rfc = ROC_rfc.fit(Taiwan_orig_valid, RfcValid_pred)

In [18]:
print('Logistic Regression classification thresholds - Validation dataset')
print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC_log.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC_log.ROC_margin)
print('')
print('Random Forest classification thresholds - Validation dataset')
print("Optimal classification threshold (with fairness constraints) = %.4f" % ROC_rfc.classification_threshold)
print("Optimal ROC margin = %.4f" % ROC_rfc.ROC_margin)

Logistic Regression classification thresholds - Validation dataset
Optimal classification threshold (with fairness constraints) = 0.7128
Optimal ROC margin = 0.0000

Random Forest classification thresholds - Validation dataset
Optimal classification threshold (with fairness constraints) = 0.7722
Optimal ROC margin = 0.0000


In [19]:
# Get Predictions from the Validation Set

In [20]:
# Metrics for the test set
fav_log_inds = LogValid_pred.scores > best_log_class_thresh
LogValid_pred.labels[fav_log_inds]  = LogValid_pred.favorable_label
LogValid_pred.labels[~fav_log_inds] = LogValid_pred.unfavorable_label
display(Markdown("#### Logistic Regression - Validation set"))
display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))
metric_valid_bef = compute_metrics(Taiwan_orig_valid, LogValid_pred, 
                                   unprivileged_groups, privileged_groups)

fav_rfc_inds = RfcValid_pred.scores > best_rfc_class_thresh
RfcValid_pred.labels[fav_rfc_inds]  = RfcValid_pred.favorable_label
RfcValid_pred.labels[~fav_rfc_inds] = RfcValid_pred.unfavorable_label
display(Markdown("#### Random Forest - Validation set"))
display(Markdown("##### Raw predictions - No fairness constraints, only maximizing balanced accuracy"))
metric_valid_bef = compute_metrics(Taiwan_orig_valid, RfcValid_pred, 
                                   unprivileged_groups, privileged_groups)

#### Logistic Regression - Validation set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Balanced accuracy = 0.6990
Statistical parity difference = 0.0669
Disparate impact = 1.0887
Average odds difference = 0.0598
Equal opportunity difference = 0.0515
Theil index = 0.1455


#### Random Forest - Validation set

##### Raw predictions - No fairness constraints, only maximizing balanced accuracy

Balanced accuracy = 0.7224
Statistical parity difference = 0.0418
Disparate impact = 1.0569
Average odds difference = 0.0184
Equal opportunity difference = 0.0348
Theil index = 0.1615


In [21]:
# Metrics for the transformed test set
LogROC_test_pred = ROC_log.predict(LogTest_pred)
display(Markdown("#### Logistic Regression - Test set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_log_test_aft = compute_metrics(Taiwan_orig_test, LogROC_test_pred, 
                                  unprivileged_groups, privileged_groups)

RfcROC_test_pred = ROC_rfc.predict(RfcTest_pred)
display(Markdown("#### Random Forest - Test set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_rfc_test_aft = compute_metrics(Taiwan_orig_test, RfcROC_test_pred, 
                                  unprivileged_groups, privileged_groups)

#### Logistic Regression - Test set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 0.6889
Statistical parity difference = 0.0399
Disparate impact = 1.0505
Average odds difference = 0.0089
Equal opportunity difference = 0.0295
Theil index = 0.1213


#### Random Forest - Test set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 0.7264
Statistical parity difference = 0.0167
Disparate impact = 1.0232
Average odds difference = -0.0121
Equal opportunity difference = -0.0027
Theil index = 0.1732


In [22]:
metric_test_bld = BinaryLabelDatasetMetric(LogROC_test_pred,
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)
print('Number of instances           :', metric_test_bld.num_instances())
print('Base Rate                     :', metric_test_bld.base_rate())
print('Consistency                   :', metric_test_bld.consistency())
print('Disparate Impact              :', metric_test_bld.disparate_impact())
print('Mean Difference               :', metric_test_bld.mean_difference())
print('Statistical Parity Difference :', metric_test_bld.statistical_parity_difference()) 

Number of instances           : 3000.0
Base Rate                     : 0.8116666666666666
Consistency                   : [0.784]
Disparate Impact              : 1.0504995778533277
Mean Difference               : 0.039919055504796686
Statistical Parity Difference : 0.039919055504796686


In [23]:
metric_test_bld = BinaryLabelDatasetMetric(RfcROC_test_pred,
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)
print('Number of instances           :', metric_test_bld.num_instances())
print('Base Rate                     :', metric_test_bld.base_rate())
print('Consistency                   :', metric_test_bld.consistency())
print('Disparate Impact              :', metric_test_bld.disparate_impact())
print('Mean Difference               :', metric_test_bld.mean_difference())
print('Statistical Parity Difference :', metric_test_bld.statistical_parity_difference()) 

Number of instances           : 3000.0
Base Rate                     : 0.7283333333333334
Consistency                   : [0.7476]
Disparate Impact              : 1.023240585950483
Mean Difference               : 0.016720677249885796
Statistical Parity Difference : 0.016720677249885796


In [24]:
Taiwan_log_ROC_test_df,d = LogROC_test_pred.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Taiwan_rfc_ROC_test_df,d = RfcROC_test_pred.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
print('Test After - Logistic Regression - DEFAULT value counts:')
print(Taiwan_log_ROC_test_df.DEFAULT.value_counts())
print('Test After - Random Forest      - DEFAULT value counts:')
print(Taiwan_rfc_ROC_test_df.DEFAULT.value_counts())

Test After - Logistic Regression - DEFAULT value counts:
1.0    2435
0.0     565
Name: DEFAULT, dtype: int64
Test After - Random Forest      - DEFAULT value counts:
1.0    2185
0.0     815
Name: DEFAULT, dtype: int64


In [25]:
# Metrics for the transformed train set -  - we expect a perfect score!!!
Taiwan_log_ROC_train = ROC_log.predict(LogTrain_pred)
display(Markdown("#### Logistic Regression - Training set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_log_train = compute_metrics(Taiwan_orig_train, Taiwan_log_ROC_train, 
                                   unprivileged_groups, privileged_groups)

Taiwan_rfc_ROC_train = ROC_rfc.predict(RfcTrain_pred)
display(Markdown("#### Random Forest - Training set"))
display(Markdown("##### Transformed predictions - With fairness constraints"))
metric_rfc_train = compute_metrics(Taiwan_orig_train, Taiwan_rfc_ROC_train, 
                                  unprivileged_groups, privileged_groups)

#### Logistic Regression - Training set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 1.0000
Statistical parity difference = 0.0340
Disparate impact = 1.0451
Average odds difference = 0.0000
Equal opportunity difference = 0.0000
Theil index = 0.0000


#### Random Forest - Training set

##### Transformed predictions - With fairness constraints

Balanced accuracy = 1.0000
Statistical parity difference = 0.0340
Disparate impact = 1.0451
Average odds difference = 0.0000
Equal opportunity difference = 0.0000
Theil index = 0.0000


In [26]:
# Get model performance for the transformed training dataset
#Train Logistic Regression and Random Forest Classifiers on original training data
X_log_train           = Taiwan_log_ROC_train.features
y_log_train           = Taiwan_log_ROC_train.labels.ravel()
X_rfc_train           = Taiwan_rfc_ROC_train.features
y_rfc_train           = Taiwan_rfc_ROC_train.labels.ravel()

UnbiasedLogModel      = LogisticRegression(random_state=101)
UnbiasedRfcModel      = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
UnbiasedLogModel.fit(X_log_train, y_log_train)
UnbiasedRfcModel.fit(X_rfc_train, y_rfc_train)
UnbiasedLogPrediction = UnbiasedLogModel.predict(X_log_train)
UnbiasedRfcPrediction = UnbiasedRfcModel.predict(X_rfc_train)

# Determine the baseline model accuracy for Logistic Regression and Random Forest Classifiers
display(Markdown("#### Unbiased training dataset - model scores"))
print('Unbiased Logistic regression validation accuracy:',UnbiasedLogModel.score(X_log_train, y_log_train))
print('Unbiased Random Forest       validation accuracy:',UnbiasedRfcModel.score(X_rfc_train, y_rfc_train))
print('')
print('Unbiased Logistic regression balanced accuracy  :', balanced_accuracy_score(y_log_train, UnbiasedLogPrediction))
print('Unbiased Random forest balanced accuracy        :', balanced_accuracy_score(y_rfc_train, UnbiasedRfcPrediction))

#### Unbiased training dataset - model scores

Unbiased Logistic regression validation accuracy: 0.7760833333333333
Unbiased Random Forest       validation accuracy: 0.8096666666666666

Unbiased Logistic regression balanced accuracy  : 0.49989264626945784
Unbiased Random forest balanced accuracy        : 0.6180156352612024


In [27]:
# Is it possible to predict the Sensitive Variable from the transformed dataset - LOG REG
Log_ROC_df, d = Taiwan_log_ROC_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False) 

X_se_train = Log_ROC_df.drop(protected_attr,axis=1)
y_se_train = Log_ROC_df[protected_attr]
X_se_test  = Taiwan_test_df.drop(protected_attr,axis=1)
y_se_test  = Taiwan_test_df[protected_attr]
Se_UnbiasedLogModel = LogisticRegression(random_state=101)
Se_UnbiasedLogModel.fit(X_se_train, y_se_train) 
yseLog_pred =  Se_UnbiasedLogModel.predict(X_se_test)
# Now test whether we can predict Gender from the test dataset
print('Model Accuracy for predicting the Sensitive Variable before bias transformation:')
print(f"Biased Logistic regression validation accuracy: {Se_UnbiasedLogModel.score(X_se_test, y_se_test)}")
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_se_test, yseLog_pred)}")

Model Accuracy for predicting the Sensitive Variable before bias transformation:
Biased Logistic regression validation accuracy: 0.5065
Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.5008655556202553


In [ ]:
# Is it possible to predict the Sensitive Variable from the transformed dataset - RFC
Rfc_ROC_df, d = Taiwan_rfc_ROC_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False) 
X_se_train = Rfc_ROC_df.drop(protected_attr,axis=1)
y_se_train = Rfc_ROC_df[protected_attr]
X_se_test  = Taiwan_test_df.drop(protected_attr,axis=1)
y_se_test  = Taiwan_test_df[protected_attr]
Se_UnbiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
Se_UnbiasedRfcModel.fit(X_se_train, y_se_train) 
yseRfc_pred =  Se_UnbiasedRfcModel.predict(X_se_test)
# Now test whether we can predict Gender from the test dataset
print('Model Accuracy for predicting the Sensitive Variable before bias transformation:')
print(f"Biased Random Forest       validation accuracy: {Se_UnbiasedRfcModel.score(X_se_test, y_se_test)}")
print('Biased Balanced accuracy')
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_se_test, yseRfc_pred)}")

Model Accuracy for predicting the Sensitive Variable before bias transformation: